## MNIST 机器学习入门
---
> 引言：编程入门有‘Hello world’ , ML 入门有‘MNIST’

### 简介
---
MNIST 是一个入门级的计算机视觉数据集，他包含：
* 各种手写数字图片（x 值 -- 试卷）
* 每一张图片对应的标签（y 值 -- 答案）



### 训练/测试数据集获取
---

In [32]:
# 执行这段 code，回去 执行 input_data.py 这个文件中的方法，download 相关的数据集
import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

### 模型构建
---
$ y = softmax(Wx + b)$

In [34]:
import tensorflow as tf

# init x , None means 第一维度可以是 任意长度 的值
x = tf.placeholder("float", [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

#  模型实现 ( y = Wx + b)
#  Wx + b 的结果输入到 tf.nn.softmax() 函数中
y = tf.nn.softmax(tf.matmul(x, W) + b)

### 模型训练
---
这个时候，到<u>**损失函数**</u>登场了

一种常见、漂亮的成本函数："交叉熵"（cross-entropy）
$$
H_y^|(y) =  -\sum_i{y^|_ilog(y_i)}
$$

In [35]:
y_ = tf.placeholder("float", [None, 10])

# 计算 交叉熵
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

有了交叉熵（loss(y)）,现在要开始优化啦，用<u>**梯度下降**</u>

In [36]:
# 梯度下降算法（gradient descent algorithm）
# 学习率 0.01
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

### 模型建立完毕，要启动啦
---

In [44]:
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    # loop 1000 times
    for i in range(1000):
        # 随机抓取训练数据中的100个批处理数据点
        batch_xs, batch_ys = mnist.train.next_batch(100)
        # 把真实值给 y_ , input数据给 x ，开始训练
        sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})
    # 评估模型
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    # bool 转 float ----> tf.cast()
    # 求平均值 -----> tf.reduct_mean()
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9214


### 评估模型
---
* `tf.argmax`：一个有用的函数，能给出tensor 对象在某一维度上的其数据最大值所在的索引值。
例如：
`tf.argmax(y, 1)`表示模型对于任一输入x预测得到的 标签值
`tf.argmax(y_, 1)`代表正确的标签
用`tf.equal` 检测我们的预测是否真实标签匹配（索引 位置一样表示匹配）